In [1]:
%load_ext autoreload
%autoreload 2

import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import pandas as pd
import re

from op_analytics.cli.subcommands.pulls.defillama.dataaccess import DefiLlama

import urllib3
import warnings
pd.set_option('display.float_format', lambda x: '%.3f' % x)
urllib3.disable_warnings()
warnings.filterwarnings("ignore")

In [3]:
PATTERNS_TO_FILTER = [
    "-borrowed",
    "-vesting",
    "-staking",
    "-pool2",
    "-treasury",
    "-cex",
    "^treasury$",
    "^borrowed$",
    "^staking$",
    "^pool2$",
    "^pool2$",
    "polygon-bridge-&-staking",  # Added this as a full match
    ".*-cex$",  # Added this to match anything ending with -cex
]

CATEGORIES_TO_FILTER = ["CEX", "Chain"]

alignment_dict = {
    "Metis": "OP Stack fork",
    "Blast": "OP Stack fork",
    "Mantle": "OP Stack fork",
    "Zircuit": "OP Stack fork",
    "RSS3": "OP Stack fork",
    "Rollux": "OP Stack fork",
    "Ancient8": "OP Stack fork",
    "Manta": "OP Stack fork",
    "Cyber": "OP Chain",
    "Mint": "OP Chain",
    "Ham": "OP Chain",
    "Polynomial": "OP Chain",
    "Lisk": "OP Chain",
    "BOB": "OP Chain",
    "Mode": "OP Chain",
    "World Chain": "OP Chain",
    "Base": "OP Chain",
    "Kroma": "OP Chain",
    "Boba": "OP Chain",
    "Fraxtal": "OP Chain",
    "Optimism": "OP Chain",
    "Shape": "OP Chain",
    "Zora": "OP Chain"
}

alignment_df = pd.DataFrame(list(alignment_dict.items()), columns=["chain", "alignment"])

token_data = [
    {"token": "ETH", "token_category": "Native Asset"},
    {"token": "WETH", "token_category": "Native Asset"},
    {"token": "SOL", "token_category": "Native Asset"},
    {"token": "wBTC", "token_category": "Wrapped Assets"},
    {"token": "cbBTC", "token_category": "Wrapped Assets"},
    {"token": "MBTC", "token_category": "Wrapped Assets"},

    {"token": "stETH", "token_category": "Liquid Staking"},
    {"token": "wstETH", "token_category": "Liquid Staking"},
    {"token": "eETH", "token_category": "Liquid Restaking"},
    {"token": "weETH", "token_category": "Liquid Restaking"},
    {"token": "sfrxETH", "token_category": "Liquid Staking"},
    {"token": "rETH", "token_category": "Liquid Staking"},
    {"token": "mETH", "token_category": "Liquid Staking"},
    {"token": "rsETH", "token_category": "Liquid Restaking"},
    {"token": "cbETH", "token_category": "Liquid Staking"},
    {"token": "ezETH", "token_category": "Liquid Restaking"},
    {"token": "rswETH", "token_category": "Liquid Restaking"},
    {"token": "swETH", "token_category": "Liquid Staking"},
    {"token": "frxETH", "token_category": "Liquid Staking"},
    {"token": "ETHX", "token_category": "Liquid Staking"},
    {"token": "lsETH", "token_category": "Liquid Staking"},
    {"token": "oETH", "token_category": "Liquid Staking"},
    {"token": "EBTC", "token_category": "Liquid Restaking"},
    {"token": "LBTC", "token_category": "Liquid Restaking"},
    {"token": "SUPEROETHB", "token_category": "Liquid Staking"},
    {"token": "WSUPEROETHB", "token_category": "Liquid Staking"},
    {"token": "TETH", "token_category": "Liquid Staking"},
    {"token": "OSETH", "token_category": "Liquid Staking"},
    {"token": "cmETH", "token_category": "Liquid Restaking"},
    {"token": "WRSETH", "token_category": "Liquid Restaking"},
    {"token": "WEETH.BASE", "token_category": "Liquid Restaking"},
    
    {"token": "USDC", "token_category": "Stablecoins"},
    {"token": "USDT", "token_category": "Stablecoins"},
    {"token": "FDUSD", "token_category": "Stablecoins"},
    {"token": "PYUSD", "token_category": "Stablecoins"},
    {"token": "TUSD", "token_category": "Stablecoins"},
    {"token": "DAI", "token_category": "Stablecoins"},
    {"token": "USDE", "token_category": "Stablecoins"},
    {"token": "USDD", "token_category": "Stablecoins"},
    {"token": "FRAX", "token_category": "Stablecoins"},
    {"token": "EURC", "token_category": "Stablecoins"},
    {"token": "AGEUR", "token_category": "Stablecoins"},
    {"token": "USDS", "token_category": "Stablecoins"},
    {"token": "USDB", "token_category": "Stablecoins"},
    {"token": "DOLA", "token_category": "Stablecoins"},
    {"token": "SUSDE", "token_category": "Stablecoins"},
    {"token": "USD0++", "token_category": "Stablecoins"},
    {"token": "USD0", "token_category": "Stablecoins"},
    {"token": "SUSD", "token_category": "Stablecoins"},
    {"token": "CRVUSD", "token_category": "Stablecoins"},
    {"token": "USDC+", "token_category": "Stablecoins"},
    {"token": "USDZ", "token_category": "Stablecoins"},
    {"token": "STAR", "token_category": "Stablecoins"},
    {"token": "USDBC", "token_category": "Stablecoins"},
    {"token": "USD+", "token_category": "Stablecoins"},
    {"token": "CDXUSD", "token_category": "Stablecoins"},
    {"token": "HYUSD", "token_category": "Stablecoins"},
    {"token": "STAR", "token_category": "Stablecoins"},
    {"token": "EURS", "token_category": "Stablecoins"},
    {"token": "AXLEUROC", "token_category": "Stablecoins"},


    # Solana Liquid staking
    {"token": "MSOL", "token_category": "Liquid Staking"},
    {"token": "JUPSOL", "token_category": "Liquid Staking"},
    {"token": "BNSOL", "token_category": "Liquid Staking"},
    {"token": "SSOL", "token_category": "Liquid Restaking"},
    {"token": "BBSOL", "token_category": "Liquid Restaking"},
    {"token": "LAINESOL", "token_category": "Liquid Staking"},
    {"token": "STSOL", "token_category": "Liquid Staking"},
    {"token": "STRONGSOL", "token_category": "Liquid Staking"},
    {"token": "HUBSOL", "token_category": "Liquid Staking"},
    {"token": "PATHSOL", "token_category": "Liquid Staking"},
    {"token": "STEPSOL", "token_category": "Liquid Staking"},
    {"token": "EDGESOL", "token_category": "Liquid Staking"},
    {"token": "JITOSOL", "token_category": "Liquid Staking"},
    {"token": "DSOL", "token_category": "Liquid Staking"},
    {"token": "BONKSOL", "token_category": "Liquid Staking"},
    {"token": "VSOL", "token_category": "Liquid Staking"},
    {"token": "HSOL", "token_category": "Liquid Staking"},
    # {"token": "ARB", "token_category": "Layer 2 Token"},
    # {"token": "OP", "token_category": "Layer 2 Token"},
    # {"token": "MODE", "token_category": "Layer 2 Token"},
]

token_categories = pd.DataFrame(token_data)

token_categories["token"] = token_categories["token"].str.upper()

- Pull this data fresh, should be okay to leave protocol metadata date as-is
- I would use "2024-11-30" as your latest date, we ran into a few data issues with more recent data
- Make sure your secrets are up to date, Pedro updated them on Dec 2nd to work with GCS
- There could be lingering data issues but Pedro addressed a bunch today

In [5]:
duckdb_client = DefiLlama.PROTOCOLS_TOKEN_TVL.read(min_date="2024-08-01")

df_protocol_tvl = duckdb_client.sql(
"""
SELECT
    dt,
    protocol_slug,
    chain,
    token,
    app_token_tvl,
    app_token_tvl_usd
FROM protocols_token_tvl_v1
""").to_df()

2024-12-02 20:06:12 [info     ] loaded vault from .env file    filename=vault.py lineno=32
2024-12-02 20:06:12 [debug    ] loaded vault: 17 items         filename=vault.py lineno=76
2024-12-02 20:06:12 [info     ] querying markers for 'protocols_token_tvl_v1' DateFilter(min_date=datetime.date(2024, 8, 1), max_date=None, datevals=None) filename=dataaccess.py lineno=202
2024-12-02 20:06:12 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=26
2024-12-02 20:06:13 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=38
2024-12-02 20:06:13 [info     ] 132 markers found              filename=dataaccess.py lineno=216
2024-12-02 20:06:15 [info     ] registered view 'protocols_token_tvl_v1' using 132 parquet paths filename=dataaccess.py lineno=228
┌────────────────────────┐
│          name          │
│        varchar         │
├────────────────────────┤
│ protocols_token_tvl_v1 │
└────────────────────────┘



In [6]:
duckdb_client = DefiLlama.PROTOCOLS_METADATA.read(min_date="2024-12-02")

df_metadata = duckdb_client.sql(
"""
SELECT 
    protocol_name,
    protocol_slug,
    protocol_category,
    parent_protocol,
    CASE WHEN misrepresented_tokens = 'True' THEN 1
        WHEN misrepresented_tokens = 'False' THEN 0
        ELSE 0
    END AS misrepresented_tokens
FROM protocols_metadata_v1
""").to_df()

2024-12-02 20:06:26 [info     ] querying markers for 'protocols_metadata_v1' DateFilter(min_date=datetime.date(2024, 12, 2), max_date=None, datevals=None) filename=dataaccess.py lineno=202
2024-12-02 20:06:26 [info     ] 2 markers found                filename=dataaccess.py lineno=216
2024-12-02 20:06:28 [info     ] registered view 'protocols_metadata_v1' using 2 parquet paths filename=dataaccess.py lineno=228
┌────────────────────────┐
│          name          │
│        varchar         │
├────────────────────────┤
│ protocols_metadata_v1  │
│ protocols_token_tvl_v1 │
└────────────────────────┘



In [8]:
# drop duplicates due to an ongoing data upload issue
df_all = pd.merge(
    df_metadata.drop_duplicates(), 
    df_protocol_tvl.drop_duplicates(), 
    on="protocol_slug",
    how="left"
)


In [9]:
# Merge data and join alignment and token categories
df_all = pd.merge(df_all, alignment_df, on="chain", how="left")
df_all["alignment"] = df_all["alignment"].fillna("Other")
df_all = pd.merge(df_all, token_categories, on="token", how="left")
df_all["token_category"] = df_all["token_category"].fillna("Other")


In [10]:
# Chain level misrepresented tokens
df_misrep = (
    df_all
    [["protocol_slug", "chain", "misrepresented_tokens", "token"]]
    .groupby(["protocol_slug", "chain", "misrepresented_tokens"])
    .agg(
        token_count=("token", "nunique"),
        has_usdt=("token", lambda x: 1 if "USDT" in x.values else 0)
    )
    .reset_index()
)

df_misrep["chain_misrepresented_tokens"] = (
    (df_misrep["misrepresented_tokens"] == 1) 
    & (df_misrep["token_count"] == 1) 
    & (df_misrep["has_usdt"] == 1)
).astype(int)

df_all = pd.merge(
    df_all, 
    df_misrep[["protocol_slug", "chain", "chain_misrepresented_tokens"]], 
    on=["protocol_slug", "chain"],
    how="left"
)

In [11]:
# remove protocols and chains

def matches_filter_pattern(s):
    return any(re.search(pattern, s, re.IGNORECASE) for pattern in PATTERNS_TO_FILTER)

df_all["chain"] = df_all["chain"].astype(str)

df_chain_protocol = df_all[["chain", "protocol_slug", "protocol_category"]].drop_duplicates()

df_chain_protocol["protocol_filters"] = (
    df_chain_protocol["chain"].apply(matches_filter_pattern)
    | (df_chain_protocol["protocol_slug"] == "polygon-bridge-&-staking")
    | df_chain_protocol["protocol_slug"].str.endswith("-cex")
    | df_chain_protocol.protocol_category.isin(CATEGORIES_TO_FILTER)
).astype(int)

# small subset for analysis, actual logic will include more (all?) chains
df_chain_protocol["chains_to_keep"] = (
    (df_all.alignment.isin(["OP Chain", "OP Stack Fork"]) 
    | df_all.chain.isin(["Ethereum", "Arbitrum", "Solana"]))
    ).astype(int)

filter_mask = (df_chain_protocol.protocol_filters == 0) & (df_chain_protocol.chains_to_keep == 1)

df_filtered = pd.merge(
    df_all,
    df_chain_protocol[filter_mask][["chain", "protocol_slug", "protocol_category"]],
    on=["chain", "protocol_slug", "protocol_category"],
    how="inner",
)



In [12]:
# misc data processing
df_filtered["dt"] = pd.to_datetime(df_filtered["dt"])
df_filtered["parent_protocol"] = df_filtered["parent_protocol"].str.replace("parent#", "")
df_filtered["token"] = df_filtered["token"].str.upper()
df_filtered["token_category"] = df_filtered["token_category"].fillna("Other")

df_filtered["token_category_misrep"] = np.where(
    (df_filtered.chain_misrepresented_tokens == 1),
    "Misrepresented TVL", 
    df_filtered.token_category
)

In [13]:
# Make some treemaps

In [17]:
def plot_nested_protocol_breakdown(data, date, chain, date_diff=90):

    data["dt"] = pd.to_datetime(data["dt"])
    target_date = pd.to_datetime(date)
    previous_date = (target_date - pd.Timedelta(days=date_diff)).strftime("%Y-%m-%d")

    filtered_data = data[
        (data["dt"] == target_date) & (data["chain"] == chain) & (data["app_token_tvl_usd"] >= 10_000)
    ]

    previous_data = data[
        (data["dt"] == previous_date) & (data["chain"] == chain)
    ]

    merged_data = filtered_data.merge(
        previous_data[[ "protocol_category", "parent_protocol", "token_category", "app_token_tvl_usd"]],
        on=[ "protocol_category", "parent_protocol", "token_category"],
        suffixes=("", "_previous"),
        how="left",
    )

    merged_data["app_token_tvl_usd_previous"].fillna(0.01, inplace=True)

    merged_data["percent_change"] = (
        (merged_data["app_token_tvl_usd"] - merged_data["app_token_tvl_usd_previous"])
        / merged_data["app_token_tvl_usd_previous"]
    ) * 100

    merged_data["percent_change"] = merged_data["percent_change"].clip(lower=-500, upper=500)

    fig = px.treemap(
        merged_data,
        path=[px.Constant("Total"), "protocol_category", "parent_protocol", "token_category"],
        values="app_token_tvl_usd", 
        color="percent_change", 
        color_continuous_scale="RdBu",
        range_color=[-100, 100],
        title=f"{chain}: Token Category <> App TVL Last {date_diff} Days",
        width=800,
        height=800,
    )

    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

    fig.show()

    return merged_data
    

In [20]:
protocol_breakdown = df_filtered.groupby(["dt", "chain", "protocol_category", "parent_protocol", "token_category_misrep"]).agg(
    {"app_token_tvl_usd": "sum"}
).reset_index().rename(columns={"token_category_misrep": "token_category"})

In [8]:
plot_df = plot_nested_protocol_breakdown(protocol_breakdown, "2024-11-20", "Solana", 30)

In [22]:
protocol_token_breakdown = df_filtered.groupby(["dt", "chain", "protocol_category", "parent_protocol", "token_category_misrep",  "token"]).agg(
    {"app_token_tvl_usd": "sum"}
).reset_index().rename(columns={"token_category_misrep": "token_category"})

In [23]:
def plot_nested_protocol_token_breakdown(data, date, chain, date_diff=90):

    data["dt"] = pd.to_datetime(data["dt"])
    target_date = pd.to_datetime(date)
    previous_date = (target_date - pd.Timedelta(days=date_diff)).strftime("%Y-%m-%d")

    filtered_data = data[
        (data["dt"] == target_date) & (data["chain"] == chain) & (data["app_token_tvl_usd"] >= 10_000)
    ]

    previous_data = data[
        (data["dt"] == previous_date) & (data["chain"] == chain)
    ]

    merged_data = filtered_data.merge(
        previous_data[[ "protocol_category", "parent_protocol", "token_category", "token", "app_token_tvl_usd"]],
        on=[ "protocol_category", "parent_protocol", "token_category", "token"],
        suffixes=("", "_previous"),
        how="left",
    )

    merged_data["app_token_tvl_usd_previous"].fillna(0.01, inplace=True)

    merged_data["percent_change"] = (
        (merged_data["app_token_tvl_usd"] - merged_data["app_token_tvl_usd_previous"])
        / merged_data["app_token_tvl_usd_previous"]
    ) * 100

    merged_data["percent_change"] = merged_data["percent_change"].clip(lower=-500, upper=500)

    fig = px.treemap(
        merged_data,
        path=[px.Constant("Total"), "protocol_category", "parent_protocol", "token_category", "token"], 
        values="app_token_tvl_usd", 
        color="percent_change",
        color_continuous_scale="RdBu",
        range_color=[-100, 100], 
        title=f"{chain}: Token Category <> App TVL Last {date_diff} Days",
        width=800,
        height=800,
    )

    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

    fig.show()

    return merged_data

In [9]:
plot_df = plot_nested_protocol_token_breakdown(protocol_token_breakdown, "2024-12-01", "Base", 7)

In [58]:


def calculate_net_flows(data, date, date_diff, agg_cols):

    target_date = pd.to_datetime(date)
    previous_date = (target_date - pd.Timedelta(days=date_diff)).strftime("%Y-%m-%d")
    
    filtered_data = data[
        (data["dt"] == target_date) & (data["app_token_tvl_usd"] >= 1_000)
    ]
    
    filtered_data_grouped = (
        filtered_data.groupby(agg_cols + ["token"])
        .agg({"app_token_tvl": "sum", "app_token_tvl_usd": "sum"})
        .reset_index()
    )
    
    # set this edge case equal to the usd value
    filtered_data_grouped.loc[(filtered_data_grouped.app_token_tvl == 0), "app_token_tvl"] = (
        filtered_data_grouped.loc[(filtered_data_grouped.app_token_tvl == 0), "app_token_tvl_usd"]
    ) 
    
    previous_data = data[
        (data["dt"] == previous_date) & (data["app_token_tvl_usd"] >= 0.01)
    ]
    
    previous_data_grouped = (
        previous_data.groupby(agg_cols + ["token"])
        .agg({"app_token_tvl": "sum"})
        .reset_index()
    )
    
    merged_data = filtered_data_grouped.merge(
        previous_data_grouped,
        on=agg_cols+["token"],
        suffixes=("", "_previous"),
        how="left",
    )
    
    merged_data["app_token_tvl_previous"] = merged_data["app_token_tvl_previous"].fillna(0.01)
    
    merged_data["app_token_tvl_previous"] = merged_data["app_token_tvl_previous"].replace(0, 0.01)
    
    merged_data["rate_usd"] = merged_data["app_token_tvl_usd"] / merged_data["app_token_tvl"]
    
    merged_data["app_token_tvl_previous_today_usd"] = merged_data["app_token_tvl_previous"] * merged_data["rate_usd"]
    
    grouped_data = (
        merged_data.groupby(agg_cols)
        .agg({"app_token_tvl_usd": "sum", "app_token_tvl_previous_today_usd": "sum"})
        .reset_index()
    )
    
    grouped_data["net_flow_usd"] = grouped_data["app_token_tvl_usd"] - grouped_data["app_token_tvl_previous_today_usd"]
    grouped_data["flow_percent_change"] = grouped_data["net_flow_usd"] / grouped_data["app_token_tvl_previous_today_usd"] * 100

    return grouped_data


In [75]:
token_category_flows = calculate_net_flows(
    df_filtered[df_filtered.chain != "Solana"],
    "2024-11-30",
    30,
    ["protocol_category", "parent_protocol"]
)

In [76]:
token_category_flows.net_flow_usd.sum()

np.float64(12095324934.04638)

In [77]:
(
token_category_flows[
    (token_category_flows.protocol_category == "Lending")]
    .sort_values(by="net_flow_usd", ascending=False)
    .head(10)
)

,protocol_category,parent_protocol,app_token_tvl_usd,app_token_tvl_previous_today_usd,net_flow_usd,flow_percent_change
592,Lending,aave,18605968966.982,17352781683.133,1253187283.849,7.222
681,Lending,spark,4527646654.915,3786525007.049,741121647.866,19.573
650,Lending,morpho,2479006216.308,1928781204.576,550225011.732,28.527
600,Lending,avalon-labs,126047910.784,38141609.118,87906301.666,230.474
617,Lending,euler,59511645.206,10501607.996,49010037.211,466.691
649,Lending,moonwell,188859461.855,167603547.174,21255914.682,12.682
712,Lending,zerolend,103648173.099,82772550.682,20875622.417,25.220
615,Lending,dolomite,71696936.634,50925503.281,20771433.353,40.788
691,Lending,termfinance,41376352.319,30619536.543,10756815.776,35.131
682,Lending,strike,14515200.027,10094053.368,4421146.659,43.800


In [78]:
# looks like some sort of migration  happened with synthetix in nov, causing this big spike 

(
token_category_flows[
    (token_category_flows.protocol_category == "Derivatives")]
    .sort_values(by="net_flow_usd", ascending=False)
    .head(10)
)

,protocol_category,parent_protocol,app_token_tvl_usd,app_token_tvl_previous_today_usd,net_flow_usd,flow_percent_change
260,Derivatives,synthetix,228516203.459,15881724.499,212634478.960,1338.863
199,Derivatives,gmx,558103368.572,504140876.190,53962492.382,10.704
270,Derivatives,wasabi,17908796.613,5774942.907,12133853.706,210.112
164,Derivatives,apex-protocol,95615578.498,89562646.475,6052932.024,6.758
239,Derivatives,paradex,18104397.713,13898250.964,4206146.749,30.264
262,Derivatives,tlx-finance,5486357.730,1944841.350,3541516.381,182.098
162,Derivatives,aevo,56130049.903,53841868.160,2288181.743,4.250
188,Derivatives,edgex,4698294.668,2575067.438,2123227.230,82.453
197,Derivatives,gains-network,28228188.602,26152584.065,2075604.538,7.937
236,Derivatives,orderly-network,18061522.342,16806010.178,1255512.164,7.471


In [80]:
(
token_category_flows[
    (token_category_flows.protocol_category.isin(["Liquid Staking", "Liquid Restaking"]))]
    .sort_values(by="net_flow_usd", ascending=False)
    .head(10)
)

,protocol_category,parent_protocol,app_token_tvl_usd,app_token_tvl_previous_today_usd,net_flow_usd,flow_percent_change
744,Liquid Restaking,magpie-ecosystem,1542428112.320,849887714.513,692540397.807,81.486
739,Liquid Restaking,ether-fi,9274195781.346,8651401397.330,622794384.016,7.199
745,Liquid Restaking,mantle-restaking,549469924.776,37.799,549469886.977,1453674033.700
759,Liquid Staking,binance-staked-eth,5510688907.005,4978488202.810,532200704.195,10.690
742,Liquid Restaking,kelp-dao,1280904315.013,908656827.700,372247487.313,40.967
750,Liquid Restaking,renzo,1548703625.715,1244544868.527,304158757.189,24.439
768,Liquid Staking,lido,35324816608.593,35139516461.726,185300146.867,0.527
790,Liquid Staking,treehouse-protocol,418404317.780,362089119.725,56315198.055,15.553
785,Liquid Staking,stakewise,497873660.272,477198343.041,20675317.231,4.333
777,Liquid Staking,redacted,120595378.858,109864416.154,10730962.704,9.767


In [81]:
(
token_category_flows[
    (token_category_flows.protocol_category.isin(["Liquid Staking", "Liquid Restaking"]))]
    .sort_values(by="app_token_tvl_usd", ascending=False)
    .head(10)
)

,protocol_category,parent_protocol,app_token_tvl_usd,app_token_tvl_previous_today_usd,net_flow_usd,flow_percent_change
768,Liquid Staking,lido,35324816608.593,35139516461.726,185300146.867,0.527
739,Liquid Restaking,ether-fi,9274195781.346,8651401397.330,622794384.016,7.199
759,Liquid Staking,binance-staked-eth,5510688907.005,4978488202.810,532200704.195,10.690
779,Liquid Staking,rocket-pool,2701430236.789,4478077248.861,-1776647012.072,-39.674
771,Liquid Staking,meth-protocol,1710670772.473,1729869513.813,-19198741.339,-1.110
750,Liquid Restaking,renzo,1548703625.715,1244544868.527,304158757.189,24.439
744,Liquid Restaking,magpie-ecosystem,1542428112.320,849887714.513,692540397.807,81.486
742,Liquid Restaking,kelp-dao,1280904315.013,908656827.700,372247487.313,40.967
746,Liquid Restaking,mellow-protocol,834216956.996,991867664.016,-157650707.020,-15.894
761,Liquid Staking,coinbase-wrapped-staked-eth,648296592.334,704837655.576,-56541063.242,-8.022


In [94]:
# I can't get this to work with multiple layers

def plot_flow_tvl(data, date, date_diff, agg_cols):
    plot_data = calculate_net_flows(
        data,
        date,
        date_diff,
        agg_cols
    )
    
    plot_data["net_flow_usd"].fillna(0, inplace=True)
    plot_data.replace([float('inf'), float('-inf')], 0, inplace=True)

    fig = px.treemap(
        plot_data[plot_data.net_flow_usd != 0],
        path=agg_cols,
        values="net_flow_usd",
        color="flow_percent_change",
        color_continuous_scale="RdBu",
        range_color=[-100, 100],
        title=f"TVL and Net Flows for EVM chains last {date_diff} days",
        width=800,
        height=800,
    )

    fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))

    fig.show()


In [96]:
plot_flow_tvl(
    df_filtered[df_filtered.chain != "Solana"], 
    "2024-11-30",
    30,
    ["protocol_category", "parent_protocol"]
)